In [7]:
import sys
sys.path.insert(0, '..')

In [8]:
import pandas as pd
import numpy as np
from sklearn.metrics import f1_score
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
from config import RANDOM_STATE, N_SPLITS
from evaluate import get_pred_labels
from helper import _process_pred_labels

In [10]:
NAME = 'keras'
oof_pred_proba = pd.read_csv(f'../oof_predictions/{NAME}/{NAME}_pred_proba.csv', index_col='review_id')
oof_pred_labels = pd.read_csv(f'../oof_predictions/{NAME}/{NAME}_pred_labels.csv', index_col='review_id')
test_pred_proba = pd.read_csv(f'../test_predictions/{NAME}/{NAME}_pred_proba.csv', index_col='review_id')
test_pred_labels = pd.read_csv(f'../test_predictions/{NAME}/{NAME}_pred_labels.csv', index_col='review_id')
target = pd.read_pickle('../data/prepared/target.pkl')

In [12]:
oof_pred_proba.head()

,0,1,2,3,4,5,6,7,8
review_id,,,,,,,,,
0,0.995651,0.000712,0.000711,0.000868,1.014354e-04,0.000564,0.000165,0.000708,0.003923
1,0.282773,0.091371,0.008451,0.011578,1.460868e-02,0.039173,0.089764,0.010368,0.532191
2,0.998825,0.000071,0.000201,0.000159,1.690472e-05,0.000205,0.000068,0.000280,0.001817
5,0.999982,0.000002,0.000028,0.000007,2.234412e-07,0.000028,0.000004,0.000020,0.000023
7,0.011999,0.001015,0.000222,0.002821,3.967544e-04,0.000546,0.000458,0.000432,0.984965


In [13]:
def get_tresholds(y_true: pd.DataFrame, y_pred: pd.DataFrame) -> pd.DataFrame:
    y_pred_labels = pd.DataFrame(np.zeros_like(y_true, dtype=np.int8), index=y_true.index, columns=y_true.columns)
    best_tresholds = {}
    for col in y_true.columns:
        candidates = [0.0] + y_pred[col].round(2).sort_values().unique().tolist() + [1.0]
        scores = [f1_score(y_true[col].values, np.where(y_pred[col].values >= tresh, 1, 0))
                  for tresh in candidates]
        idx = np.argmax(scores)
        best_score = scores[idx]
        best_treshold = candidates[idx]
        best_tresholds[col] = best_treshold
        y_pred_labels.loc[:, col] = np.where(y_pred[col] >= best_treshold, 1, 0)
        print(f'col {col}, best_score {best_score}, best_treshold {best_treshold}')
    
    print('f1 samples', f1_score(y_true, y_pred_labels, average='samples', zero_division=0))
    return best_tresholds

In [14]:
def get_tresholds2(y_true: pd.DataFrame, y_pred: pd.DataFrame) -> pd.DataFrame:
    candidates = np.unique(oof_pred_proba.round(3).values)
    scores = [f1_score(y_true.values, np.where(y_pred.values >= tresh, 1, 0), average='samples', zero_division=0)
              for tresh in candidates]
    idx = np.argmax(scores)
    best_score = scores[idx]
    best_treshold = candidates[idx]
    print(f'best_score {best_score}, best_treshold {best_treshold}')

    return best_treshold

In [15]:
best_treshold = get_tresholds2(target, oof_pred_proba)

best_score 0.84057708939382, best_treshold 0.329


In [16]:
# 0.8394803050554289
# 0.84057708939382

In [19]:
test_one_tresh_labels = pd.DataFrame(data=np.zeros_like(test_pred_proba, dtype=np.int8), columns=test_pred_proba.columns, index=test_pred_proba.index)
test_one_tresh_labels.loc[:, :] = np.where(test_pred_proba.values >= best_treshold, 1, 0)
test_one_tresh_labels.head()

,0,1,2,3,4,5,6,7,8
review_id,,,,,,,,,
3,0,0,0,0,0,0,0,0,1
4,1,0,0,0,0,0,0,0,1
13,0,0,0,0,0,0,0,0,1
14,1,0,0,0,0,0,0,0,0
17,1,0,0,0,0,0,0,0,1


In [20]:
(test_one_tresh_labels
.apply(_process_pred_labels, axis=1)
.rename('target')
.to_csv('../submitions/keras.csv'))

In [23]:
import re

In [22]:
def __clean_numbers(x):
        if bool(re.search(r'\d', x)):
            x = re.sub('[0-9]{5,}', '#####', x)
            x = re.sub('[0-9]{4}', '####', x)
            x = re.sub('[0-9]{3}', '###', x)
            x = re.sub('[0-9]{2}', '##', x)
        return x

In [25]:
t = pd.Series(['ееёё ё 123фываё12'])
t.apply(__clean_numbers)

0    ееёё ё ###фываё##
dtype: object

In [26]:
import string

In [27]:
string.ascii_letters

'abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ'

In [32]:
s = 'фы'

In [34]:
s.

False

In [36]:
handcrafted = pd.read_pickle('../data/handcrafted/train.pkl')

In [37]:
handcrafted

,positive_cnt_!,positive_shr_!,positive_cnt_*,positive_shr_*,positive_cnt_.,positive_shr_.,"positive_cnt_,","positive_shr_,",positive_cnt_?,positive_shr_?,...,cnt_upper_chars_positive,shr_upper_chars_positive,cnt_upper_chars_negative,shr_upper_chars_negative,cnt_unique_words_positive,shr_unique_words_positive,cnt_unique_words_negative,shr_unique_words_negative,max_shr_val,max_cnt_val
review_id,,,,,,,,,,,,,,,,,,,,,
0,0,0.0,6,0.025751,4,0.017167,1,0.004292,0,0.0,...,4,0.017167,1,0.083333,31,0.911765,2,1.000000,1.00000,35
1,0,0.0,0,0.000000,0,0.000000,1,0.018519,0,0.0,...,1,0.018519,7,0.020772,6,1.000000,41,0.872340,1.00000,41
2,0,0.0,0,0.000000,2,0.006873,4,0.013746,0,0.0,...,4,0.013746,1,0.166667,48,0.857143,1,1.000000,1.00000,48
5,0,0.0,6,0.030612,6,0.030612,3,0.015306,0,0.0,...,6,0.030612,1,0.250000,29,0.966667,1,1.000000,1.00000,33
7,0,0.0,0,0.000000,1,0.033333,0,0.000000,0,0.0,...,2,0.066667,1,0.034483,4,1.000000,5,1.000000,1.00000,17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
192815,0,0.0,0,0.000000,0,0.000000,0,0.000000,0,0.0,...,1,0.032258,6,0.127660,3,0.375000,2,0.166667,1.00000,12
192818,0,0.0,0,0.000000,1,0.025000,0,0.000000,0,0.0,...,1,0.025000,1,0.027027,4,1.000000,5,1.000000,1.00000,22
192820,0,0.0,0,0.000000,1,0.035714,0,0.000000,0,0.0,...,1,0.035714,2,0.057143,3,1.000000,4,1.000000,1.00000,20


In [38]:
morph = pd.read_pickle('../data/morph/train.pkl')

In [39]:
morph.head()

,positive_normalized,positive_tags,negative_normalized,negative_tags
review_id,,,,
0,недавно устроиться в ****** на должность кладо...,"ADVB, VERB perf masc indc sing past intr, PREP...",повысить з/п,"INFN perf tran,"
1,"расположение офиса, свой род стабильность (гос...","NOUN inan nomn neut sing, , ADJF gent neut sin...",упростить процесс - всё процесс ради процессов...,"INFN perf tran, NOUN inan nomn masc plur, , PR..."
2,в наш банк всё по закон в план зарплата и граф...,"PREP, ADJF loct masc sing, NOUN inan loct masc...",ничего,ADVB
5,я в ****** около год работать оператором. норм...,"NPRO nomn sing 1per, PREP, , PREP, NOUN inan g...",нет.,
7,в принцип норм.работать можно,"PREP, NOUN inan loct masc sing, INFN impf intr...",всегда есть к что стремиться,"ADVB, INFN impf tran, PREP, NPRO datv neut sin..."


In [40]:
from tensorflow.keras import layers

In [41]:
def get_encoders(data: pd.DataFrame, **keywords):
    encoders = {}
    for col in data.columns:
        encoder = layers.TextVectorization(max_tokens=10000, name=col, **keywords)
        encoder.adapt(data[col])
        encoders[col] = encoder
    return encoders

In [ ]:
encoders = get_encoders(train[TEXT_COLS+UNORDERED_CATEGORIES])